# QR Factorisation

We will look at two different ways for computing the QR factorisation of a matrix. To goal is to start from a matrix $A$ and write it as the product of an orthogonal matrix $Q$ and an upper-triangular matrix $R$.

In [ ]:
import numpy as np
from numpy.linalg import norm

A = np.array([[1,2,3],[4,5,6],[7,8,10]])
b = np.array([1,2,3])
n = 3

Numpy has a built-in function for doing this. Let's use it to check what the answer should be:

In [ ]:
(Q,R) = np.linalg.qr(A)
(Q,R)

## Gram-Schmidt orthogonalization

The first approach will be to transform the vectors in the columns of $A$ to a set of orthogonal vectores using the Gram-Schmidt approach. The basic idea of Gram-Schmidt is to build up an orthonormal set of vectors by projecting out non-orthogonal pieces. The following image illustrates this.
![Gram-Schmidt Visualisation](Gram-Schmidt_orthonormalization_process.gif "Gram-Schmidt Visualisation")
Let's now implement this with our test matrix $A$.

First, we construct three vectors $a_1$, $a_2$ and $a_3$ from the columns of $A$.

In [ ]:
(a1, a2, a3) = np.transpose(A)

Now, our first orthonormal vector is just $a_1$ normalised to have length 1:

In [ ]:
u1 = a1
e1 = u1 / norm(u1)

To construct our second orthonormal vector, let's start with $a_2$, project out the part along the $a_1$ direction and normalise the result:

In [ ]:
u2 = a2 - (e1@a2)*e1
e2 = u2/norm(u2)

Let's also project this piece out from $a_3$ now (this is not essential, but helps improve the numerical stability of the algorithm)

In [ ]:
u3 = a3 - (e1@a3)*e1

To construct our third orthonormal vector, let's project out the part along the previous two directions and normalise the result:

In [ ]:
u3 = u3 - (e2@a3)*e2
e3 = u3 / norm(u3)

Now we have our three orthogonal vectors, we can put them into the columns of Q

In [ ]:
Q = np.transpose([e1, e2, e3])

To get $R$, we note that $A = Q R$ means that $Q^T A = Q^T Q R = R$ since $Q$ is an orthogonal matrix. Let's use this to compute $R$:

In [ ]:
R = np.transpose(Q)@A

In [ ]:
Q

In [ ]:
R

As expected, $R$ is (almost) an upper-triangular matrix. It is only __almost__ upper triangular because floating point arithmetic is not exact.

## Using Householder reflections

The Gram-Schmidt process can be a very effective way to orthogonalise a set of vectors, but it does run into problems with numerical stability. This can happen, for example, in the case where we are starting with vectors that are nearly linearly dependent. Then we would be subtracting two large vectors to produce one small one, and we know that this is a recipe for disaster with floating point arithmetic.

One way around this problem is to use a different approach to orthogonalization. A very popular method uses the idea of Householder reflections. These take a vector $x$ and reflect it about a plane defined by another vector $v$:
![Householder reflection](Householder.png "Householder reflection")
This is clearly equivalent to multiplying $x$ by the __Householder reflection matrix__
$$ H = I - 2 \frac{v v^T}{||v||^2}$$
Note that $H$ is a *symmetric, orthogonal matrix*.

Now, if we choose $v$ appropriately then we can use it to zero out below the pivot in each column, thus producing $R$. In particular, if
$$v = a - ||a|| e_k$$ then $H a = |a| e_k$ so if $e_k$ is a unit vector in the $k$-th direction this does exactly what we want to the column $a$.

Let's now implement this in practice.

First, we work on the first column of $A$.

In [ ]:
u1 = a1 - (-np.sign(a1[0]))*norm(a1)*np.array([1,0,0])
v1 = u1/norm(u1)
H1 = np.identity(3) - 2*np.outer(v1,v1)

In [ ]:
A1 = H1@A

and the second column

In [ ]:
a2 = A1[1:,1]
u2 = a2 - (-np.sign(a2[0]))*norm(a2)*np.array([1,0])
v2 = u2/norm(u2)
H2 = np.identity(3)
H2[1:,1:] -= 2*np.outer(v2,v2)

In [ ]:
A2 = H2@A1

and finally the third column

In [ ]:
a3 = A2[2:,2]
u3 = a3 - (-np.sign(a3[0]))*norm(a3)*np.array([1])
v3 = u3/norm(u3)
H3 = np.identity(3)
H3[2:,2:] -= 2*np.outer(v3,v3)

In [ ]:
H3

In [ ]:
A3 = H3@A2

In [ ]:
A3

We now have transformed to exactly $R$.

In [ ]:
R

And we can easily construct $Q$ from the Householder matrices

In [ ]:
np.transpose(H3@H2@H1)

In [ ]:
Q